## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.


## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import os

## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [2]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'
df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))
df.head()

The shape of the dataset is (6407, 16).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


In [3]:
# remove minutes and seconds from timestamp
df['timestamp'] = df['timestamp'].map(lambda x: str(x)[: -6])
df.head(2)

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19


We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

In [4]:
weather_df = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))
print("The shape of the dataset is {}.\n\n".format(weather_df.shape))

weather_df.head()

The shape of the dataset is (6901, 12).




,Year,Day,Month,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
0,2020,27,7,18,Fair,64.0,0.0,64.0,70.0,20.0,10.0,No
1,2017,30,9,17,Partly Cloudy,NaN,NaN,71.1,57.0,9.2,10.0,No
2,2017,27,6,5,Overcast,NaN,NaN,57.9,87.0,15.0,9.0,No
3,2016,7,9,9,Clear,NaN,NaN,66.9,73.0,4.6,10.0,No
4,2019,19,10,2,Fair,52.0,0.0,52.0,89.0,0.0,9.0,No


In [5]:
weather_df['timestamp'] = pd.to_datetime(weather_df[["Year", "Month", "Day", "Hour"]])
weather_df['timestamp'] = weather_df['timestamp'].map(lambda x: str(x)[: -6])

weather_df.head()

,Year,Day,Month,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected,timestamp
0,2020,27,7,18,Fair,64.0,0.0,64.0,70.0,20.0,10.0,No,2020-07-27 18
1,2017,30,9,17,Partly Cloudy,NaN,NaN,71.1,57.0,9.2,10.0,No,2017-09-30 17
2,2017,27,6,5,Overcast,NaN,NaN,57.9,87.0,15.0,9.0,No,2017-06-27 05
3,2016,7,9,9,Clear,NaN,NaN,66.9,73.0,4.6,10.0,No,2016-09-07 09
4,2019,19,10,2,Fair,52.0,0.0,52.0,89.0,0.0,9.0,No,2019-10-19 02


In [6]:
weather_df.describe()

,Year,Day,Month,Hour,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
count,6901.000000,6901.000000,6901.000000,6901.000000,3292.000000,3574.000000,6899.000000,6899.000000,6556.000000,6900.000000
mean,2018.293001,15.624837,6.775250,12.789886,59.762515,0.006444,59.905465,68.570952,10.813164,9.441851
std,1.390524,8.703753,3.567982,5.874155,9.338456,0.028822,7.977860,16.180126,6.406376,1.640513
min,2016.000000,1.000000,1.000000,0.000000,31.100000,0.000000,36.000000,10.000000,0.000000,0.000000
25%,2017.000000,8.000000,4.000000,8.000000,54.000000,0.000000,54.000000,59.000000,5.800000,10.000000
50%,2019.000000,15.000000,7.000000,14.000000,59.000000,0.000000,59.000000,70.000000,10.000000,10.000000
75%,2020.000000,23.000000,10.000000,17.000000,66.000000,0.000000,64.900000,80.000000,15.000000,10.000000
max,2020.000000,31.000000,12.000000,23.000000,98.000000,0.490000,98.100000,100.000000,40.300000,10.000000


In [7]:
weather_df.isna().sum()

Year                    0
Day                     0
Month                   0
Hour                    0
Weather_Condition       1
Wind_Chill(F)        3609
Precipitation(in)    3327
Temperature(F)          2
Humidity(%)             2
Wind_Speed(mph)       345
Visibility(mi)          1
Selected                0
timestamp               0
dtype: int64

In [8]:
from sklearn import preprocessing

weather_df['Weather_Condition'] = preprocessing.LabelEncoder().fit_transform(list(weather_df['Weather_Condition']))

In [9]:
# Drop Weather_Condition and Wind_Chill(F) 
weather_df = weather_df.drop(columns=['Wind_Chill(F)','Precipitation(in)'])

weather_df = weather_df.groupby('timestamp').mean().reset_index()

print("The shape of the dataset is {}.\n\n".format(weather_df.shape))

The shape of the dataset is (5972, 10).




In [10]:
weather_df = weather_df.fillna(weather_df.mean())

weather_df.isna().sum()

timestamp            0
Year                 0
Day                  0
Month                0
Hour                 0
Weather_Condition    0
Temperature(F)       0
Humidity(%)          0
Wind_Speed(mph)      0
Visibility(mi)       0
dtype: int64

In [11]:
df = pd.merge(df, weather_df,
                        how="left", on=["timestamp"])

print("The shape of the dataset is {}.\n\n".format(df.shape))
df.head()

The shape of the dataset is (6407, 25).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,...,timestamp,Year,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,...,2016-03-25 15,2016.0,25.0,3.0,15.0,19.5,65.0,56.0,20.15,10.0
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,...,2020-05-05 19,2020.0,5.0,5.0,19.0,14.5,58.0,80.5,21.00,10.0
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,...,2016-09-16 19,2016.0,16.0,9.0,19.0,0.0,62.1,80.0,9.20,10.0
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,...,2020-03-29 19,2020.0,29.0,3.0,19.0,3.0,58.0,70.0,10.00,10.0
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,...,2019-10-09 08,2019.0,9.0,10.0,8.0,3.0,58.0,65.0,3.00,10.0


In [12]:
df.drop(columns=['ID', 'Year', 'Month', 'Day', 'Hour']).describe()

,Lat,Lng,Distance(mi),Severity,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
count,6407.000000,6407.000000,6407.000000,6407.000000,6371.000000,6371.000000,6371.000000,6371.000000,6371.000000
mean,37.765653,-122.405990,0.135189,2.293429,11.093824,59.970375,68.127209,10.867929,9.440505
std,0.032555,0.028275,0.396360,0.521225,6.740421,7.926291,16.071629,6.219420,1.606963
min,37.609619,-122.510440,0.000000,1.000000,0.000000,36.000000,10.000000,0.000000,0.120000
25%,37.737096,-122.412210,0.000000,2.000000,3.000000,54.500000,59.000000,6.000000,10.000000
50%,37.768238,-122.404835,0.000000,2.000000,14.000000,59.000000,70.000000,10.400000,10.000000
75%,37.787813,-122.392477,0.041000,3.000000,17.000000,65.000000,80.000000,15.000000,10.000000
max,37.825626,-122.349734,6.820000,4.000000,26.000000,98.000000,100.000000,40.300000,10.000000


In [13]:
import xml.etree.ElementTree as ET

xml_data = ET.parse(os.path.join(dataset_path, 'holidays.xml'))
root = xml_data.getroot()

get_range = lambda col: range(len(col))
holiday_list = [{r[i].tag:r[i].text for i in get_range(r)} for r in root]

xml_holidays_df = pd.DataFrame.from_dict(holiday_list)
print("The shape of the dataset is {}.\n\n".format(xml_holidays_df.shape))

xml_holidays_df.head()

The shape of the dataset is (90, 2).




,date,description
0,2012-01-02,New Year Day
1,2012-01-16,Martin Luther King Jr. Day
2,2012-02-20,Presidents Day (Washingtons Birthday)
3,2012-05-28,Memorial Day
4,2012-07-04,Independence Day


In [14]:
xml_holidays_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         90 non-null     object
 1   description  90 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


In [15]:
df['date'] = df['timestamp'].map(lambda x: str(x)[:-3])

In [16]:
df = pd.merge(df, xml_holidays_df,
                        how="left", on=["date"])

print("The shape of the dataset is {}.\n\n".format(df.shape))
df.head()

The shape of the dataset is (6407, 27).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,...,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),date,description
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,...,25.0,3.0,15.0,19.5,65.0,56.0,20.15,10.0,2016-03-25,NaN
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,...,5.0,5.0,19.0,14.5,58.0,80.5,21.00,10.0,2020-05-05,NaN
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,...,16.0,9.0,19.0,0.0,62.1,80.0,9.20,10.0,2016-09-16,NaN
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,...,29.0,3.0,19.0,3.0,58.0,70.0,10.00,10.0,2020-03-29,NaN
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,...,9.0,10.0,8.0,3.0,58.0,65.0,3.00,10.0,2019-10-09,NaN


In [17]:
df["description"] = df["description"].notna().astype(int)
df.head(2)

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,...,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),date,description
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,...,25.0,3.0,15.0,19.5,65.0,56.0,20.15,10.0,2016-03-25,0
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,...,5.0,5.0,19.0,14.5,58.0,80.5,21.00,10.0,2020-05-05,0


In [18]:
df = df.fillna(df.mean())

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6407 entries, 0 to 6406
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 6407 non-null   int64  
 1   Lat                6407 non-null   float64
 2   Lng                6407 non-null   float64
 3   Bump               6407 non-null   bool   
 4   Distance(mi)       6407 non-null   float64
 5   Crossing           6407 non-null   bool   
 6   Give_Way           6407 non-null   bool   
 7   Junction           6407 non-null   bool   
 8   No_Exit            6407 non-null   bool   
 9   Railway            6407 non-null   bool   
 10  Roundabout         6407 non-null   bool   
 11  Stop               6407 non-null   bool   
 12  Amenity            6407 non-null   bool   
 13  Side               6407 non-null   object 
 14  Severity           6407 non-null   int64  
 15  timestamp          6407 non-null   object 
 16  Year               6407 

In [20]:
from scipy import stats

# z_scores = stats.zscore(df[['Lat','Lng']])
# abs_z_scores = np.abs(z_scores)
# filtered_entries = (abs_z_scores < 3).all(axis=1)
# df = df[filtered_entries]

#df.shape

In [21]:
df = df.dropna()

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6407 entries, 0 to 6406
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 6407 non-null   int64  
 1   Lat                6407 non-null   float64
 2   Lng                6407 non-null   float64
 3   Bump               6407 non-null   bool   
 4   Distance(mi)       6407 non-null   float64
 5   Crossing           6407 non-null   bool   
 6   Give_Way           6407 non-null   bool   
 7   Junction           6407 non-null   bool   
 8   No_Exit            6407 non-null   bool   
 9   Railway            6407 non-null   bool   
 10  Roundabout         6407 non-null   bool   
 11  Stop               6407 non-null   bool   
 12  Amenity            6407 non-null   bool   
 13  Side               6407 non-null   object 
 14  Severity           6407 non-null   int64  
 15  timestamp          6407 non-null   object 
 16  Year               6407 

In [23]:
corr = df.corr()
corr.style.background_gradient(cmap=plt.cm.Reds)

/opt/conda/lib/python3.7/site-packages/pandas/io/formats/style.py:1264: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
/opt/conda/lib/python3.7/site-packages/pandas/io/formats/style.py:1265: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Severity,Year,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),description
ID,1.000000,0.012043,0.009135,nan,0.024395,0.010125,-0.021899,0.007417,-0.009012,0.004945,nan,0.003892,0.003176,0.020703,-0.006317,-0.001147,0.010157,-0.014049,0.013261,-0.013030,0.014994,-0.009102,-0.028050,0.007551
Lat,0.012043,1.000000,0.388177,nan,0.039760,0.040971,0.011296,0.012027,0.010412,-0.002388,nan,0.390530,0.088219,0.099581,0.001705,0.005389,-0.032178,0.021040,0.000425,0.038522,0.012465,-0.053336,-0.001539,-0.007717
Lng,0.009135,0.388177,1.000000,nan,0.074003,-0.123170,-0.032626,0.191309,-0.000733,-0.030877,nan,0.385575,-0.102261,0.145313,-0.015943,-0.007216,-0.019339,0.008745,-0.000012,0.026294,0.018404,-0.067904,-0.028971,-0.026672
Bump,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Distance(mi),0.024395,0.039760,0.074003,nan,1.000000,-0.020309,-0.007164,-0.028275,0.000341,-0.033987,nan,-0.054533,-0.033825,-0.013141,0.033515,0.008797,0.076754,0.002128,0.012276,-0.018432,0.001301,-0.021479,-0.001638,0.007248
Crossing,0.010125,0.040971,-0.123170,nan,-0.020309,1.000000,0.072222,-0.160848,-0.003744,0.430823,nan,-0.037446,0.319284,-0.090314,0.013836,0.024445,-0.007482,-0.037262,-0.018388,-0.040346,0.014686,-0.038847,0.012011,0.018897
Give_Way,-0.021899,0.011296,-0.032626,nan,-0.007164,0.072222,1.000000,-0.012378,-0.000270,0.041317,nan,0.041475,-0.004251,-0.012186,-0.001035,-0.004935,0.011874,0.007975,0.006139,0.012861,-0.019756,-0.013383,0.007558,-0.003305
Junction,0.007417,0.012027,0.191309,nan,-0.028275,-0.160848,-0.012378,1.000000,-0.007145,-0.094416,nan,0.075290,-0.089347,-0.068328,0.075379,-0.023077,-0.039373,0.017529,-0.020730,0.013291,-0.031359,-0.004427,-0.010450,0.002472
No_Exit,-0.009012,0.010412,-0.000733,nan,0.000341,-0.003744,-0.000270,-0.007145,1.000000,-0.002063,nan,-0.004111,-0.002454,-0.007034,-0.021869,-0.015259,-0.006047,0.000288,0.005403,0.007872,-0.002048,0.007116,0.004363,-0.001908
Railway,0.004945,-0.002388,-0.030877,nan,-0.033987,0.430823,0.041317,-0.094416,-0.002063,1.000000,nan,-0.034703,0.126759,-0.033322,0.036845,0.020675,0.020315,-0.010157,-0.042327,-0.001688,-0.024435,-0.009385,0.003255,0.007329


## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [24]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID', 'Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID', 'Severity'])
y_val = val_df['Severity']


As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

In [25]:
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
X_train = X_train[['Year', 'Stop', 'Lat', 'Lng', 'Distance(mi)', 'Crossing', 'Junction',  'Weather_Condition', 'Humidity(%)', 'Temperature(F)', 'description']]
X_val = X_val[['Year', 'Stop', 'Lat', 'Lng', 'Distance(mi)', 'Crossing', 'Junction',  'Weather_Condition', 'Humidity(%)',  'Temperature(F)', 'description']]

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [26]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

Now let's test our classifier on the validation dataset and see the accuracy.

In [27]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.7472698907956318


Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [28]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58


In [29]:
test_df['timestamp'] = test_df['timestamp'].map(lambda x: str(x)[: -6])

In [30]:
test_df = pd.merge(test_df, weather_df,
                        how="left", on=["timestamp"])

print("The shape of the dataset is {}.\n\n".format(test_df.shape))
test_df.head()

The shape of the dataset is (1601, 24).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,...,timestamp,Year,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,...,2016-04-04 19,2016.0,4.0,4.0,19.0,17.000000,63.0,60.0,10.400000,10.000000
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,...,2020-10-28 11,2020.0,28.0,10.0,11.0,3.000000,65.0,56.0,5.000000,9.000000
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,...,2019-09-09 07,2019.0,9.0,9.0,7.0,10.333333,58.0,88.0,11.666667,8.666667
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,...,2019-08-06 15,2019.0,6.0,8.0,15.0,3.000000,72.0,59.0,17.000000,10.000000
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,...,2018-10-17 09,2018.0,17.0,10.0,9.0,22.000000,57.0,77.0,5.800000,10.000000


In [31]:
test_df['date'] = test_df['timestamp'].map(lambda x: str(x)[:-3])

In [32]:
test_df = pd.merge(test_df, xml_holidays_df,
                        how="left", on=["date"])

print("The shape of the dataset is {}.\n\n".format(test_df.shape))
test_df.head()

The shape of the dataset is (1601, 26).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,...,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),date,description
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,...,4.0,4.0,19.0,17.000000,63.0,60.0,10.400000,10.000000,2016-04-04,NaN
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,...,28.0,10.0,11.0,3.000000,65.0,56.0,5.000000,9.000000,2020-10-28,NaN
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,...,9.0,9.0,7.0,10.333333,58.0,88.0,11.666667,8.666667,2019-09-09,NaN
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,...,6.0,8.0,15.0,3.000000,72.0,59.0,17.000000,10.000000,2019-08-06,NaN
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,...,17.0,10.0,9.0,22.000000,57.0,77.0,5.800000,10.000000,2018-10-17,NaN


Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [33]:
test_df["description"] = test_df["description"].notna().astype(int)
df.head(2)

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,...,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),date,description
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,...,25.0,3.0,15.0,19.5,65.0,56.0,20.15,10.0,2016-03-25,0
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,...,5.0,5.0,19.0,14.5,58.0,80.5,21.00,10.0,2020-05-05,0


In [34]:
test_df = test_df.fillna(test_df.mean())

In [35]:
X_test = test_df.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
X_test = X_test[['Year', 'Stop', 'Lat', 'Lng', 'Distance(mi)', 'Crossing', 'Junction', 'Weather_Condition', 'Humidity(%)', 'Temperature(F)', 'description']]

y_test_predicted = classifier.predict(X_test)

test_df['Severity'] = y_test_predicted

test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,...,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),date,description,Severity
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,...,4.0,19.0,17.000000,63.0,60.0,10.400000,10.000000,2016-04-04,0,2
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,...,10.0,11.0,3.000000,65.0,56.0,5.000000,9.000000,2020-10-28,0,2
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,...,9.0,7.0,10.333333,58.0,88.0,11.666667,8.666667,2019-09-09,0,2
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,...,8.0,15.0,3.000000,72.0,59.0,17.000000,10.000000,2019-08-06,0,2
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,...,10.0,9.0,22.000000,57.0,77.0,5.800000,10.000000,2018-10-17,0,2


Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [36]:
test_df[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.